In [18]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit
import marginal_effects as me


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sother', 
    'sage',
    'sempl', 
    'sincome',
    'spop', 
    'sbehavior',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    'omajother', 
    'osplit', 
    
    # Encounter characteristics
    'daytime',
    'inctype_lin', 
    'year'
]


In [20]:
# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',

    # Officer vars (white is reference)
    #'omajblack', --> Udelukkes fordi der ikke er nogen
    'omajhisp',
    #'omajother',

    # Encounter vars
    'daytime'
]

In [21]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [22]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))
y.shape

Shape x: (3799, 6)
Rank x: 6


(3799,)

In [23]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

LPM results
Dependent variable: anyuseofforce_coded

R2 = 0.004
sigma2 = nan


,b_hat,se,t
sblack,0.0048,0.0042,1.1465
shisp,0.0126,0.0063,1.9881
smale,0.0068,0.0022,3.0771
sage,-0.0000,0.0000,-0.4581
omajhisp,0.0042,0.0107,0.3885
daytime,-0.0010,0.0022,-0.4525


In [24]:
theta0 = probit.starting_values(y, x)
theta0.ndim==1

True

In [25]:
ll = probit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll), -1.0411283428047824)

False

In [26]:
#probit_results = est.estimate(probit.q, theta0, y, x)

In [27]:
#probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
#probit_tab

In [28]:
theta0 = logit.starting_values(y, x)
theta0 

array([ 1.90417771e-02,  5.02863186e-02,  2.73331524e-02, -7.37894881e-05,
        1.66389576e-02, -4.05348732e-03])

In [29]:
ll = logit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll),-0.9974267061091704)

False

In [30]:
logit_results = est.estimate(logit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.033326
         Iterations: 65
         Function evaluations: 574
         Gradient evaluations: 82


In [31]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 65 iter. (574 func. evals.). Final criterion:  0.03333.
Logit, y = anyuseofforce_coded


,theta,se,t
sblack,0.1378,0.7223,0.1907
shisp,0.6087,0.5319,1.1443
smale,0.0021,0.6597,0.0032
sage,-0.1628,0.0168,-9.6694
omajhisp,0.3236,1.3030,0.2484
daytime,-0.9576,0.5766,-1.6607


In [32]:
# Reference covariate profile used for marginal effects
x_ref = np.array([0, 0, 1, 25, 1, 1], dtype=float)
pd.DataFrame(x_ref.reshape(1, -1), columns=x_lab, index=['x_ref'])


,sblack,shisp,smale,sage,omajhisp,daytime
x_ref,0.0,0.0,1.0,25.0,1.0,1.0


In [33]:
# Delta-method partial effects for logit across all regressors
b_lg = logit_results['theta']
cov_lg = logit_results['cov']

binary_vars = ['sblack', 'shisp', 'smale', 'omajhisp', 'daytime']
continuous_vars = ['sage']

me_rows = []

for var in binary_vars:
    idx = x_lab.index(var)
    x0 = x_ref.copy()
    x1 = x_ref.copy()
    x0[idx] = 0.0
    x1[idx] = 1.0
    effect, se = me.discrete_effect_delta(b_lg, cov_lg, x0, x1, logit.G)
    me_rows.append({
        'Var': var,
        'Effect (LPM)': ols_results['b_hat'][idx],
        'Effect (Logit)': effect,
        's.e. (Logit)': se,
    })

for var in continuous_vars:
    idx = x_lab.index(var)
    effect, se = me.continuous_effect_delta(b_lg, cov_lg, x_ref, idx, logit.Gprime)
    me_rows.append({
        'Var': var,
        'Effect (LPM)': ols_results['b_hat'][idx],
        'Effect (Logit)': effect,
        's.e. (Logit)': se,
    })

me_tab = pd.DataFrame(me_rows).set_index('Var')
me_tab['t (Logit)'] = me_tab['Effect (Logit)'] / me_tab['s.e. (Logit)']


In [34]:
me_tab.round(4)


,Effect (LPM),Effect (Logit),s.e. (Logit),t (Logit)
Var,,,,
sblack,0.0048,0.0013,0.0074,0.1778
shisp,0.0126,0.0074,0.0091,0.8156
smale,0.0068,0.0000,0.0059,0.0032
omajhisp,0.0042,0.0025,0.0115,0.2139
daytime,-0.0010,-0.0141,0.0244,-0.5773
sage,-0.0000,-0.0015,0.0020,-0.7393
